In [1]:
from pathlib import Path

import pandas as pd
from lightning import pytorch as pl
import ray
from ray import tune
from ray.train import CheckpointConfig, RunConfig, ScalingConfig
from ray.train.lightning import (RayDDPStrategy, RayLightningEnvironment,
                                 RayTrainReportCallback, prepare_trainer)
from ray.train.torch import TorchTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import FIFOScheduler

from chemprop import data, featurizers, models, nn

/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-11 23:30:45,988	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-11 23:30:46,275	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-11 23:30:46,665	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
chemprop_dir = Path.cwd().parent
train_path = "train_data_parp1.csv"
test_path = "test_data_parp1.csv"
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['value'] # list of names of the columns containing targets

hpopt_save_dir = Path.cwd() / "hpopt_parp1_zinc" # directory to save hyperopt results
hpopt_save_dir.mkdir(exist_ok=True)

In [3]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [4]:
# Extract SMILES and target values
train_smis = df_train[smiles_column].values
train_ys = df_train[target_columns].values

test_smis = df_test[smiles_column].values
test_ys = df_test[target_columns].values

# Convert data to MoleculeDatapoint format
train_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(train_smis, train_ys)]
test_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(test_smis, test_ys)]


In [5]:
# Initialize featurizer
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

# Create training dataset
train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

# Split validation set from training data (80% train, 20% validation)
split_idx = int(len(train_data) * 0.8)
val_data = train_data[split_idx:]
train_data = train_data[:split_idx]

# Create validation and test datasets
val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data, featurizer)

In [6]:
def train_model(config, train_dset, val_dset, num_workers, scaler):

    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth)
    agg = nn.MeanAggregation()
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    ffn = nn.RegressionFFN(output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers)
    batch_norm = True
    metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=20, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, train_loader, val_loader)

In [7]:
search_space = {
    "depth": tune.qrandint(lower=2, upper=6, q=1),
    "ffn_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
    "ffn_num_layers": tune.qrandint(lower=1, upper=3, q=1),
    "message_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
}

In [8]:
ray.init()

scheduler = FIFOScheduler()

# Scaling config controls the resources used by Ray
scaling_config = ScalingConfig(
    num_workers=1,
    use_gpu=False, # change to True if you want to use GPU
)

# Checkpoint config controls the checkpointing behavior of Ray
checkpoint_config = CheckpointConfig(
    num_to_keep=1, # number of checkpoints to keep
    checkpoint_score_attribute="val_loss", # Save the checkpoint based on this metric
    checkpoint_score_order="min", # Save the checkpoint with the lowest metric value
)

run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    storage_path=hpopt_save_dir / "ray_results", # directory to save the results
)

ray_trainer = TorchTrainer(
    lambda config: train_model(
        config, train_dset, val_dset, num_workers, scaler
    ),
    scaling_config=scaling_config,
    run_config=run_config,
)

search_alg = HyperOptSearch(
    n_initial_points=1, # number of random evaluations before tree parzen estimators
    random_state_seed=42,
)

# OptunaSearch is another search algorithm that can be used
# search_alg = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="val_loss",
    mode="min",
    num_samples=2, # number of trials to run
    scheduler=scheduler,
    search_alg=search_alg,
    trial_dirname_creator=lambda trial: str(trial.trial_id), # shorten filepaths

)

tuner = tune.Tuner(
    ray_trainer,
    param_space={
        "train_loop_config": search_space,
    },
    tune_config=tune_config,
)

# Start the hyperparameter search
results = tuner.fit()

(TorchTrainer pid=18480) Started distributed worker processes: 
(TorchTrainer pid=18480) - (node_id=8cc5897c288ebf199fe9365a5a88974f9998b21b1dae5473da94721a, ip=10.233.0.20, pid=18714) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=18714) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=18714) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/train/lightning/_lightning_utils.py:262: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
(RayTrainWorker pid=18714) `get_trial_name` is deprecated because the concept of a `Trial` will soon be removed in Ray Train.Ray Train will no longer assume that it's running within a Ray Tune `Trial` in the future. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment vari

Epoch 0:  68%|██████▊   | 19/28 [00:05<00:02,  3.21it/s, v_num=3.41e+7, train_loss_step=0.358]


(TorchTrainer pid=18715) Started distributed worker processes: 
(TorchTrainer pid=18715) - (node_id=8cc5897c288ebf199fe9365a5a88974f9998b21b1dae5473da94721a, ip=10.233.0.20, pid=18951) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=18951) Setting up process group for: env:// [rank=0, world_size=1]


Epoch 0: 100%|██████████| 28/28 [00:08<00:00,  3.28it/s, v_num=3.41e+7, train_loss_step=0.398]
Validation: |          | 0/? [00:00<?, ?it/s]
(RayTrainWorker pid=18714) 
Validation DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]
(RayTrainWorker pid=18714) 
Validation DataLoader 0:  17%|█▋        | 1/6 [00:00<00:01,  4.71it/s]
(RayTrainWorker pid=18714) 
Validation DataLoader 0:  33%|███▎      | 2/6 [00:00<00:00,  4.14it/s]
(RayTrainWorker pid=18714) 
Validation DataLoader 0:  50%|█████     | 3/6 [00:00<00:00,  4.23it/s]
(RayTrainWorker pid=18714) 
Validation DataLoader 0:  67%|██████▋   | 4/6 [00:00<00:00,  4.33it/s]
(RayTrainWorker pid=18714) 
Validation DataLoader 0:  83%|████████▎ | 5/6 [00:01<00:00,  4.33it/s]
(RayTrainWorker pid=18714) 
Epoch 0: 100%|██████████| 28/28 [00:09<00:00,  2.83it/s, v_num=3.41e+7, train_loss_step=0.398, val_loss=0.753]


(RayTrainWorker pid=18714) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_parp1_zinc/ray_results/TorchTrainer_2025-03-11_23-31-58/ade62d14/checkpoint_000000)


...

(RayTrainWorker pid=18951) `Trainer.fit` stopped: `max_epochs=20` reached.


Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  83%|████████▎ | 5/6 [00:01<00:00,  4.47it/s] [repeated 5x across cluster]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]
(RayTrainWorker pid=18714) 
(RayTrainWorker pid=18714) 
(RayTrainWorker pid=18714) 
(RayTrainWorker pid=18714) 
(RayTrainWorker pid=18714) 
(RayTrainWorker pid=18714) 
Epoch 18: 100%|██████████| 28/28 [00:08<00:00,  3.17it/s, v_num=3.41e+7, train_loss_step=0.108, val_loss=0.143, train_loss_epoch=0.129]


(RayTrainWorker pid=18714) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_parp1_zinc/ray_results/TorchTrainer_2025-03-11_23-31-58/ade62d14/checkpoint_000018) [repeated 2x across cluster]


Validation DataLoader 0:  83%|████████▎ | 5/6 [00:00<00:00,  5.61it/s] [repeated 5x across cluster]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 19: 100%|██████████| 28/28 [00:07<00:00,  3.99it/s, v_num=3.41e+7, train_loss_step=0.141, val_loss=0.143, train_loss_epoch=0.122] [repeated 3x across cluster]
(RayTrainWorker pid=18714) 
(RayTrainWorker pid=18714) 
Validation DataLoader 0:  33%|███▎      | 2/6 [00:00<00:00,  5.88it/s] [repeated 2x across cluster]
(RayTrainWorker pid=18714) 
(RayTrainWorker pid=18714) 
(RayTrainWorker pid=18714) 
(RayTrainWorker pid=18714) 
Validation DataLoader 0: 100%|██████████| 6/6 [00:01<00:00,  5.96it/s]
(RayTrainWorker pid=18714) 
Epoch 19: 100%|██████████| 28/28 [00:08<00:00,  3.36it/s, v_num=3.41e+7, train_loss_step=0.141, val_loss=0.0909, train_loss_epoch=0.112]


(RayTrainWorker pid=18714) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_parp1_zinc/ray_results/TorchTrainer_2025-03-11_23-31-58/ade62d14/checkpoint_000019)
(RayTrainWorker pid=18714) `Trainer.fit` stopped: `max_epochs=20` reached.
2025-03-11 23:35:49,990	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_parp1_zinc/ray_results/TorchTrainer_2025-03-11_23-31-58' in 0.0063s.
2025-03-11 23:35:49,995	INFO tune.py:1041 -- Total run time: 231.48 seconds (231.42 seconds for the tuning loop).
(TorchTrainer pid=18480) *** SIGSEGV received at time=1741732550 on cpu 71 ***
(TorchTrainer pid=18480) PC: @     0x7f7f8c7458fe  (unknown)  ray::gcs::TaskInfoAccessor::AsyncAddTaskEventData()
(TorchTrainer pid=18480)     @     0x7f7f8da3e6f0       3680  (unknown)
(TorchTrainer pid=18480)     @     0x7f7f8c626ab5       1392 

In [9]:
results

ResultGrid<[
  Result(
    metrics={'train_loss': 0.11183732748031616, 'train_loss_step': 0.14122967422008514, 'val/rmse': 0.30153706669807434, 'val/mae': 0.2347082942724228, 'val_loss': 0.09092459827661514, 'train_loss_epoch': 0.11183732748031616, 'epoch': 19, 'step': 560},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_parp1_zinc/ray_results/TorchTrainer_2025-03-11_23-31-58/ade62d14',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_parp1_zinc/ray_results/TorchTrainer_2025-03-11_23-31-58/ade62d14/checkpoint_000019)
  ),
  Result(
    metrics={'train_loss': 0.08938580751419067, 'train_loss_step': 0.10487531870603561, 'val/rmse': 0.256196528673172, 'val/mae': 0.20003533363342285, 'val_loss': 0.06563665717840195, 'train_loss_epoch': 0.08938580751419067, 'epoch': 19, 'step': 560},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_parp1_zinc/ray_results/TorchTrain

In [10]:
result_df = results.get_dataframe()
result_df

,train_loss,train_loss_step,val/rmse,val/mae,val_loss,train_loss_epoch,epoch,step,timestamp,checkpoint_dir_name,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/train_loop_config/depth,config/train_loop_config/ffn_hidden_dim,config/train_loop_config/ffn_num_layers,config/train_loop_config/message_hidden_dim,logdir
0,0.111837,0.141230,0.301537,0.234708,0.090925,0.111837,19,560,1741732548,checkpoint_000019,...,18480,cpusrv10.scidom.de,10.233.0.20,220.536170,20,2,2000,2,500,ade62d14
1,0.089386,0.104875,0.256197,0.200035,0.065637,0.089386,19,560,1741732533,checkpoint_000019,...,18715,cpusrv10.scidom.de,10.233.0.20,195.991913,20,2,2200,2,400,41678f15


In [11]:
# best configuration
best_result = results.get_best_result()
best_config = best_result.config
best_config['train_loop_config']

{'depth': 2,
 'ffn_hidden_dim': 2200,
 'ffn_num_layers': 2,
 'message_hidden_dim': 400}

In [12]:
# best model checkpoint path
best_result = results.get_best_result()
best_checkpoint_path = Path(best_result.checkpoint.path) / "checkpoint.ckpt"
print(f"Best model checkpoint path: {best_checkpoint_path}")

Best model checkpoint path: /ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_parp1_zinc/ray_results/TorchTrainer_2025-03-11_23-31-58/41678f15/checkpoint_000019/checkpoint.ckpt


In [13]:
mpnn = models.MPNN.load_from_checkpoint(best_checkpoint_path)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=400, bias=False)
    (W_h): Linear(in_features=400, out_features=400, bias=False)
    (W_o): Linear(in_features=472, out_features=400, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=400, out_features=2200, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=2200, bias=True)
      )
      (2): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]

In [14]:
import torch
test_loader = data.build_dataloader(test_dset, shuffle=False)
with torch.inference_mode():
    trainer = pl.Trainer(
        logger=None,
        enable_progress_bar=True,
        accelerator="cpu",
        devices=1
    )
    test_preds = trainer.predict(mpnn, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
SLURM auto-requeueing enabled. Setting signal handlers.
/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 16.78it/s]


In [15]:
import numpy as np
test_preds = np.concatenate(test_preds, axis=0)
df_test['preds'] = test_preds
df_test

,smiles,value,preds
0,COC(=O)c1ccc(-c2ccc(/C=N\Nc3ccc([N+](=O)[O-])c...,10.0,8.813208
1,Cc1nc2ccccc2c(=O)n1NC(=O)c1ccc([N+](=O)[O-])cc1,9.7,9.905311
2,N#C[C@]1(NC(=O)c2ccc(Cl)c(N3CCNC3=O)c2)CCSC1,7.7,8.524395
3,COc1cccc(C[NH+]2CCC[C@H](c3ncc[nH]3)C2)c1OC(F)F,7.0,7.537811
4,CC(=O)N[C@H]1[C@H](O)[C@@H]2OC(C)(C)OC[C@H]2O[...,9.8,7.302305
...,...,...,...
193,CCNC(=O)[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[C@@H](...,7.4,7.029051
194,O=C1C[C@@H](c2cccc(Oc3ccccc3)c2)c2cc3c(cc2N1)OCO3,10.5,10.595282
195,O=C(c1nn(-c2cccc(Br)c2)c2c1CCCC2)N1CCC[C@@H]2C...,10.4,10.091125
196,C[C@@H](CCc1ccccc1)NC(=O)Nc1cc(-n2cnnn2)ccc1F,9.5,8.969509


In [16]:
from sklearn.metrics import r2_score, mean_squared_error

# Get true values (ground truth) and predictions
y_true = df_test['value'].values  # True target values
y_pred = df_test['preds'].values  # Predicted values

# Calculate metrics
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)

print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.4f}")

R² Score: 0.7278
MSE: 0.3715
